<a href="https://www.kaggle.com/code/realshaktigupta/mnist-classifier-99-4-accuracy?scriptVersionId=135483235" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
import tensorflow as tf
from tensorflow import keras

/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: libtensorflow_io.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so: undefined symbol: _ZTVN10tenso

In [3]:
(X_train_full, y_train_full), (X_test, y_test) = keras.datasets.mnist.load_data()
X_train, X_valid = X_train_full[:-5000], X_train_full[-5000:]
y_train, y_valid = y_train_full[:-5000], y_train_full[-5000:]

X_mean = X_train.mean(axis=0, keepdims=True)
X_std = X_train.std(axis=0, keepdims=True) + 1e-7
X_train = (X_train - X_mean) / X_std
X_valid = (X_valid - X_mean) / X_std
X_test = (X_test - X_mean) / X_std

X_train = X_train[..., np.newaxis]
X_valid = X_valid[..., np.newaxis]
X_test = X_test[..., np.newaxis]

11490434/11490434 [==============================] - 0s 0us/step


In [4]:
strategy = tf.distribute.MirroredStrategy()

In [5]:
with strategy.scope():
    model=keras.models.Sequential()
    model.add(tf.keras.layers.InputLayer(input_shape=(28,28,1)))
    model.add(tf.keras.layers.Conv2D(32,kernel_size=3,padding='same',
                                     activation='relu'))
    model.add(tf.keras.layers.Conv2D(64,kernel_size=3,padding='same',activation='relu'))
    model.add(tf.keras.layers.MaxPool2D())
    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.Conv2D(16,kernel_size=3,padding='same',
                                     activation='relu'))
    model.add(tf.keras.layers.Conv2D(32,kernel_size=3,padding='same',activation='relu'))
    model.add(tf.keras.layers.MaxPool2D())
    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.Flatten())
    model.add(tf.keras.layers.Dropout(0.25))
    model.add(tf.keras.layers.Dense(128,activation='relu'))
    model.add(tf.keras.layers.Normalization())
    model.add(tf.keras.layers.Dropout(0.5))
    model.add(tf.keras.layers.Dense(64,activation='relu'))
    model.add(tf.keras.layers.Dense(10,activation='softmax'))
    model.compile(optimizer="Adam",loss='sparse_categorical_crossentropy',metrics=['accuracy'])

In [6]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 28, 28, 32)        320       
                                                                 
 conv2d_1 (Conv2D)           (None, 28, 28, 64)        18496     
                                                                 
 max_pooling2d (MaxPooling2D  (None, 14, 14, 64)       0         
 )                                                               
                                                                 
 batch_normalization (BatchN  (None, 14, 14, 64)       256       
 ormalization)                                                   
                                                                 
 conv2d_2 (Conv2D)           (None, 14, 14, 16)        9232      
                                                                 
 conv2d_3 (Conv2D)           (None, 14, 14, 32)        4

In [7]:
with strategy.scope():
    checkpoint= tf.keras.callbacks.ModelCheckpoint(filepath='/kaggle/working/mnist_classifier10.h5',save_weights_only=False,
                                                   monitor='val_accuracy',save_best_only=True,save_freq="epoch",)
    early_stopping= tf.keras.callbacks.EarlyStopping(monitor='val_loss',patience=10,restore_best_weights=True)
    def lr_scheduler(epoch,lr,epochs=75):
        initial=1e-3
        if epoch<epochs*0.08:
            return initial
        elif epoch>epochs*0.08 and epoch<epochs*0.20:
            lr*=tf.math.exp(-0.1)
            return lr
        else:
            lr*=tf.math.exp(-0.008)
            return lr
    lr_scheduling=tf.keras.callbacks.LearningRateScheduler(lr_scheduler)

In [8]:
with strategy.scope():
    history = model.fit(X_train,y_train,epochs=75,
                        validation_data=(X_valid,y_valid),use_multiprocessing=True,
                        callbacks=[checkpoint,early_stopping,lr_scheduling])

Epoch 1/75
1719/1719 [==============================] - 43s 15ms/step - loss: 0.2517 - accuracy: 0.9254 - val_loss: 0.0584 - val_accuracy: 0.9840 - lr: 0.0010
Epoch 2/75
1719/1719 [==============================] - 25s 15ms/step - loss: 0.1010 - accuracy: 0.9713 - val_loss: 0.0531 - val_accuracy: 0.9858 - lr: 0.0010
Epoch 3/75
1719/1719 [==============================] - 25s 15ms/step - loss: 0.0808 - accuracy: 0.9763 - val_loss: 0.0335 - val_accuracy: 0.9904 - lr: 0.0010
Epoch 4/75
1719/1719 [==============================] - 24s 14ms/step - loss: 0.0661 - accuracy: 0.9812 - val_loss: 0.0388 - val_accuracy: 0.9908 - lr: 0.0010
Epoch 5/75
1719/1719 [==============================] - 25s 14ms/step - loss: 0.0561 - accuracy: 0.9842 - val_loss: 0.0432 - val_accuracy: 0.9904 - lr: 0.0010
Epoch 6/75
1719/1719 [==============================] - 25s 14ms/step - loss: 0.0535 - accuracy: 0.9845 - val_loss: 0.0378 - val_accuracy: 0.9894 - lr: 0.0010
Epoch 7/75
1719/1719 [========================

In [9]:
model=tf.keras.saving.load_model("/kaggle/working/mnist_classifier10.h5")

In [10]:
model.evaluate(X_test, y_test)

313/313 [==============================] - 1s 3ms/step - loss: 1068.5398 - accuracy: 0.9949


[1068.539794921875, 0.9948999881744385]